In [10]:
import pandas as pd
import numpy as np
import math
import urllib
import requests
import asyncio
from bs4 import BeautifulSoup as BS

In [60]:
#async functions

def snag_shit_pandas(url):
    response = pd.read_html(url)
    return response

async def snag_shit_soups(url):
    page = requests.get(url_list[j])
    soup = BS(page.content, 'lxml')
    response = soup.find_all('a') 
    return await response


#asynchronous  info
# Get a reference to the event loop
loop = asyncio.get_event_loop()

# Create the list of shit we want to get asnyc
requests = [asyncio.ensure_future(snag_shit_pandas(someDumbassURL)),
            asyncio.ensure_future(snag_shit_pandas(someOtherDumbShitURL))]

# Run the requests
responses = loop.run_until_complete(asyncio.gather(*requests))

responses

#found this dope shit too. add it if ou want to quit long stuff
try:
    loop.run_forever()
except KeyboardInterrupt:
    # Canceling pending tasks and stopping the loop
    asyncio.gather(*asyncio.Task.all_tasks()).cancel()
    # Stopping the loop
    loop.stop()
    # Received Ctrl+C
    loop.close() 

In [ ]:
#--------

In [11]:
#inputs stuff

tickers = ['OPHT', 'MNTA']
#tickers = ['MNTA', 'AON', 'OPHT', 'IIVI', 'MMM', 'GE']
filing_input_type = 'DEF+14'
table_var = "Summary Compensation Table"
period = 2

In [12]:
url_list = []

for i in range(0, len(tickers)):
    url_list.append('https://www.sec.gov/cgi-bin/browse-edgar?CIK=' + tickers[i] + '&owner=include&action=getcompany&type=' + filing_input_type)    
url_list

['https://www.sec.gov/cgi-bin/browse-edgar?CIK=OPHT&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=MNTA&owner=include&action=getcompany&type=DEF+14']

In [30]:
#BS grabs links here
tag_list = []
document_links = []

for j in range(0,len(url_list)):
    page = requests.get(url_list[j])
    soup = BS(page.content, 'lxml')
    doc_link = soup.find_all('a')
    count = 0

    for i in range(0,len(doc_link)):
        x = doc_link[i].get('href')
        y = x.split('/')
        if len(y) > 1:
            if (y[1] == 'Archives' and count < period):
                tag_list.append([tickers[j],x])
                count = count + 1

    for k in range(0,len(tag_list)):
        index_page = requests.get('https://www.sec.gov'+tag_list[k][1])
        soup2= BS(index_page.content, 'lxml')
        doc_link2 = soup2.find_all('a')
        if tag_list[k][0] == tickers[j]:
            document_links.append([tickers[j],doc_link2[8].get('href')])
    
document_links

[['OPHT',
  '/Archives/edgar/data/1410939/000141093918000020/proxystatementfor2018annua.htm'],
 ['OPHT',
  '/Archives/edgar/data/1410939/000104746917002856/a2231921zdef14a.htm'],
 ['MNTA',
  '/Archives/edgar/data/1235010/000104746918003290/a2235381zdef14a.htm'],
 ['MNTA',
  '/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm']]

In [31]:
filing_reader = []
for i in range(0,len(document_links)):
    url = 'https://www.sec.gov'+document_links[i][1]
    filing_reader.append([document_links[i][0],pd.read_html(url)])

In [135]:
filing_reader[2][1][21]


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Name and Principal Position,NaN,Year,NaN,Salary ($),NaN,Stock Awards(1) ($),NaN,Option Awards(1) ($),NaN,...,NaN,Total ($),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Craig A. Wheeler,NaN,NaN,2017.0,NaN,NaN,750000,NaN,NaN,1743625,...,NaN,NaN,480000,NaN,NaN,34551.0,NaN,NaN,4784639.0,NaN
3,"President, Chief Executive",NaN,NaN,2016.0,NaN,NaN,689179,NaN,NaN,2149290,...,NaN,NaN,554789,NaN,NaN,29948.0,NaN,NaN,4161024.0,NaN
4,Officer and Director,NaN,NaN,2015.0,NaN,NaN,691484,NaN,NaN,3064900,...,NaN,NaN,512723,NaN,NaN,29503.0,NaN,NaN,5344662.0,NaN
5,Scott M. Storer,NaN,NaN,2017.0,NaN,NaN,420000,NaN,NaN,—,...,NaN,NaN,137760,NaN,NaN,10848.0,NaN,NaN,568608.0,NaN
6,Senior Vice President and Chief,NaN,NaN,2016.0,NaN,NaN,40385,NaN,NaN,684000,...,NaN,NaN,—,NaN,NaN,150.0,NaN,NaN,1664910.0,NaN
7,Financial Officer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Ganesh V. Kaundinya,NaN,NaN,2017.0,NaN,NaN,448517,NaN,NaN,425068,...,NaN,NaN,196800,NaN,NaN,13481.0,NaN,NaN,1516939.0,NaN
9,Chief Operating Officer and,NaN,NaN,2016.0,NaN,NaN,433350,NaN,NaN,791487,...,NaN,NaN,201833,NaN,NaN,13091.0,NaN,NaN,1661106.0,NaN


In [131]:
tester = []
interim = filing_reader[2][1][21].dropna(how = 'all', axis = 1)
searchParameters = ("Name|Officer|Position|Salary|Stock|Award|Option|Non‑Equity Incentive Plan|Compensation|All Other|Total|Year")


for i in range(0,interim.shape[1]):
    tester.append(interim.iloc[i].str.contains(searchParameters).sum())


sum(tester)
#interim

12

In [138]:
filing_reader[0][1]

[                                              0  \
 0                                           NaN   
 1                                           NaN   
 2                     Filed by the Registrant x   
 3  Filed by a Party other than the Registrant o   
 4                    Check the appropriate box:   
 5                                             o   
 6                                             o   
 7                                             x   
 8                                             o   
 9                                             o   
 
                                                    1  
 0                                                NaN  
 1                                                NaN  
 2                                                NaN  
 3                                                NaN  
 4                                                NaN  
 5                        Preliminary Proxy Statement  
 6  Confidential, for Use of the C

In [140]:
#test section

highest_score = 0
searchParameters = ("Name|Officer|Position|Salary|Stock|Award|Option|Non‑Equity Incentive Plan|Compensation|All Other|Total|Year")
bestmatches = []
count = 0

for filings in range(0, len(filing_reader)): #for all filings
    for year in range(0, len(filing_reader[filings][1])): #for each table
        frame_counter = 0
        tester = []
        count = count + 1
        #now were in the filing
        
count

581

In [132]:
#need to fix

highest_score = 0
searchParameters = ("Name|Officer|Position|Salary|Stock|Award|Option|Non‑Equity Incentive Plan|Compensation|All Other|Total|Year")
bestmatches = []

for filings in range(0, len(filing_reader)): #for all filings
    for year in range(0, len(filing_reader[filings][1])): #for each table
        frame_counter = 0
        tester = []
        
        #now were in the filing
        
        for j in range(0,len(filing_reader[filings][1][i])): #for each column 
            interim = filing_reader[filings][1][i][j].dropna(how = 'all', axis = 1)
            tester.append(interim.iloc[i].str.contains(searchParameters).sum())
            score = sum(tester)
            
            if score > highest_score:
                highest_score = score
                bestmatch = filing_reader[filings][1][i][j]
                
        bestmatches.append([filing_reader[filings][0],bestmatch])
            
            
    
        

ValueError: No axis named 1 for object type <class 'pandas.core.series.Series'>

In [ ]:
bestmatches

In [ ]:
#stuff I cut from previous loop

if any(s in filing_reader[filings][1][i][j] for s in searchParameters):
                frame_counter += 1
            if (frame_counter > highestScore):
                bestmatch = filing_reader[filings][1][i][j]
                highestScore = frame_counter
        bestmatches.append([filing_reader[filings][0],bestmatch])
